In [1]:
import json

with open('flow_result.json', 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [2]:
with open('flow_test.json','r') as f:
    gt = json.load(f)

In [3]:
print(len(datas))
print(len(gt))
print(gt[0])

292
292
{'input_prompt': 'In a directed graph, the nodes are numbered from 0 to 10, and the edges are:\nan edge from node 0 to node 10 with capacity 1,\nan edge from node 3 to node 0 with capacity 17,\nan edge from node 3 to node 2 with capacity 15,\nan edge from node 4 to node 0 with capacity 13,\nan edge from node 4 to node 2 with capacity 14,\nan edge from node 5 to node 1 with capacity 7,\nan edge from node 5 to node 4 with capacity 16,\nan edge from node 6 to node 7 with capacity 19,\nan edge from node 6 to node 10 with capacity 3,\nan edge from node 6 to node 1 with capacity 18,\nan edge from node 6 to node 8 with capacity 10,\nan edge from node 7 to node 8 with capacity 4,\nan edge from node 7 to node 3 with capacity 15,\nan edge from node 8 to node 7 with capacity 18,\nan edge from node 8 to node 0 with capacity 7,\nan edge from node 8 to node 4 with capacity 19,\nan edge from node 8 to node 2 with capacity 14,\nan edge from node 9 to node 1 with capacity 8,\nan edge from node 

In [4]:
import re
import io
import sys
print(len(gt))
print(len(datas))

292
292


In [5]:
print(gt[2]['input_prompt'])
print(datas[2]['code'])
print(gt[2]['answer'])


In a directed graph, the nodes are numbered from 0 to 6, and the edges are:
an edge from node 0 to node 6 with capacity 3,
an edge from node 0 to node 1 with capacity 4,
an edge from node 0 to node 4 with capacity 7,
an edge from node 0 to node 5 with capacity 2,
an edge from node 1 to node 6 with capacity 8,
an edge from node 1 to node 0 with capacity 6,
an edge from node 1 to node 4 with capacity 9,
an edge from node 2 to node 6 with capacity 7,
an edge from node 2 to node 0 with capacity 5,
an edge from node 2 to node 3 with capacity 5,
an edge from node 3 to node 4 with capacity 2,
an edge from node 3 to node 5 with capacity 1,
an edge from node 4 to node 6 with capacity 7,
an edge from node 4 to node 3 with capacity 6,
an edge from node 5 to node 0 with capacity 9,
an edge from node 5 to node 1 with capacity 6,
an edge from node 5 to node 4 with capacity 2,
an edge from node 6 to node 0 with capacity 10,
an edge from node 6 to node 3 with capacity 7,
an edge from node 6 to node 2 

In [8]:
error_cnt = 0
correct_cnt = 0
for i, data in enumerate(datas):
    text = data['code']
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    ans = gt[i]['answer']

    # 创建一个 StringIO 对象来捕获输出
    output = io.StringIO()

    # 重定向 sys.stdout
    old_stdout = sys.stdout
    sys.stdout = output

    try:
        exec(code, globals(), globals())
    except:
        sys.stdout = old_stdout
        #print(code)
        error_cnt = error_cnt + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    # 获取输出内容
    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    # print(output_content)
    
    
    # print(f'llm={output_content} gt={ans}')
        
    if output_content.strip() == str(ans).split('.')[0].strip():
        correct_cnt = correct_cnt + 1
        # print(gt[int(i)]['input_prompt'])
        # print(code)
    else:
        print(gt[int(i)]['input_prompt'])
        print(output_content,ans)
        print(code)

In a directed graph, the nodes are numbered from 0 to 16, and the edges are:
an edge from node 0 to node 1 with capacity 19,
an edge from node 0 to node 8 with capacity 13,
an edge from node 0 to node 7 with capacity 1,
an edge from node 0 to node 2 with capacity 19,
an edge from node 1 to node 9 with capacity 5,
an edge from node 1 to node 14 with capacity 5,
an edge from node 1 to node 5 with capacity 17,
an edge from node 2 to node 7 with capacity 19,
an edge from node 2 to node 0 with capacity 12,
an edge from node 2 to node 4 with capacity 5,
an edge from node 3 to node 10 with capacity 11,
an edge from node 3 to node 6 with capacity 17,
an edge from node 3 to node 16 with capacity 18,
an edge from node 4 to node 1 with capacity 3,
an edge from node 4 to node 3 with capacity 17,
an edge from node 4 to node 9 with capacity 14,
an edge from node 4 to node 0 with capacity 1,
an edge from node 4 to node 11 with capacity 4,
an edge from node 4 to node 5 with capacity 12,
an edge from n

In [9]:
print(error_cnt)
print(correct_cnt)

25
217


In [10]:
print(correct_cnt/len(gt))

0.7431506849315068


In [13]:
import re
import networkx as nx

# Graph description
graph = """
Graph[name="maximum-flow"] {
    node_list = [0, 1, 2, 3, 4, 5, 6, 7];
    edge_list = [(0 -> 6)[capacity=1], (1 -> 3)[capacity=10], (3 -> 7)[capacity=3], (4 -> 2)[capacity=3], (4 -> 1)[capacity=6], (4 -> 6)[capacity=8], (5 -> 2)[capacity=3], (5 -> 3)[capacity=9], (6 -> 2)[capacity=6], (6 -> 3)[capacity=8], (7 -> 4)[capacity=6], (7 -> 0)[capacity=5]];
}
"""

# Regular expression to extract edges with weights
pattern = r'\((\d+)->(\d+)\)[capacity=(\d+)]'
matches = re.findall(pattern, graph)

# Create an undirected graph
G = nx.Graph()
print(G)
# Add edges with weights to the graph
for node1, node2, weight in matches:
    G.add_edge(int(node1), int(node2), weight=int(weight))

# Calculate maximum flow from node 7 to node 3
flow_value, flow_dict = nx.maximum_flow(G, 6, 3, capacity='weight')
print(flow_value)


Graph with 0 nodes and 0 edges


NetworkXError: node 6 not in graph